In [32]:
# all that's needed
# !pip install transformers

import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
tqdm.pandas()



In [33]:
# Load the data
df = pd.read_csv("data/rusentitweet_full.csv")
df.head()

,Unnamed: 0,text,label,id
0,0,@varlamov @McFaul На,skip,1327934765807308801
1,1,велл они всё равно что мусор так что ничего с...,negative,1252943181387350017
2,2,"""трезвая жизнь какая-то такая стрёмная""\r\n(с)...",negative,1323610669061677056
3,3,Ой какие неожиданные результаты 🤭 https://t.co...,neutral,1336231661160247297
4,4,@Shvonder_chief @dimsmirnov175 На заборе тоже ...,neutral,1292421736454127617


In [34]:
# change label column to int value
def change_label(label):
    match label:
        case 'skip':
            return 0
        case 'positive':
            return 1
        case 'neutral':
            return 2
        case 'speech':
            return 3
        case 'negative':
            return 4

df['label'] = df['label'].apply(change_label)

# print all unique labels
print(df['label'].unique())

[0 4 2 3 1]


In [35]:
# function to clean the text
import regex as re
# in this case we will remove usernames, hashtags, links and emojis
# also we will remove all punctuation
# and we will lowercase the text
def clean_text(text):
    text = text.lower()
    # remove usernames
    text = re.sub(r'@\w+', '', text)
    # remove hashtags
    text = re.sub(r'#\w+', '', text)
    # remove links
    text = re.sub(r'http\S+', '', text)
    # remove emojis
    text = re.sub(r'\\x\S+', '', text)
    # remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    return text

In [36]:
# prepare the data
df['text'] = df['text'].apply(clean_text)
df.head()

,Unnamed: 0,text,label,id
0,0,на,0,1327934765807308801
1,1,велл они всё равно что мусор так что ничего с...,4,1252943181387350017
2,2,трезвая жизнь какаято такая стрёмная\r\nс артё...,4,1323610669061677056
3,3,ой какие неожиданные результаты,2,1336231661160247297
4,4,на заборе тоже написаноа там другоео сборе д...,2,1292421736454127617


In [37]:
# tokenazation of the text
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

In [38]:
def tokenize(text):
    res = tokenizer.encode_plus(
        text,
        max_length=128,
        pad_to_max_length=True,
        truncation=True,
        return_tensors='pt'
    )
    return pd.Series([res['input_ids'], res['attention_mask']])

df[['input_ids', 'attention_mask']] = df['text'].progress_apply(tokenize)
df.head()

  0%|          | 0/13392 [00:00<?, ?it/s]

c:\Users\sarta\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


,Unnamed: 0,text,label,id,input_ids,attention_mask
0,0,на,0,1327934765807308801,"[[tensor(101), tensor(1469), tensor(102), tens...","[[tensor(1), tensor(1), tensor(1), tensor(0), ..."
1,1,велл они всё равно что мусор так что ничего с...,4,1252943181387350017,"[[tensor(101), tensor(12044), tensor(864), ten...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
2,2,трезвая жизнь какаято такая стрёмная\r\nс артё...,4,1323610669061677056,"[[tensor(101), tensor(67121), tensor(1637), te...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
3,3,ой какие неожиданные результаты,2,1336231661160247297,"[[tensor(101), tensor(32589), tensor(19201), t...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
4,4,на заборе тоже написаноа там другоео сборе д...,2,1292421736454127617,"[[tensor(101), tensor(1469), tensor(37272), te...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."


In [39]:
# split the data
test_size = 0.2
batch_size = 32

train_df, test_df = train_test_split(
    df,
    test_size=test_size,
    # shuffle=True,
    random_state=42,
    stratify=df['label'].values
)

train_set = TensorDataset(
    torch.cat(list(train_df["input_ids"].values), dim = 0),
    torch.cat(list(train_df["attention_mask"].values), dim = 0),
    torch.tensor(train_df["label"].values)
)

test_set = TensorDataset(
    torch.cat(list(test_df["input_ids"].values), dim = 0),
    torch.cat(list(test_df["attention_mask"].values), dim = 0),
    torch.tensor(test_df["label"].values)
)

train_dataloader = DataLoader(
    train_set, 
    batch_size=batch_size, 
    shuffle=True
)

test_dataloader = DataLoader(
    test_set, 
    batch_size=batch_size, 
    shuffle=True
)

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [41]:
# model first
bert_model = BertForSequenceClassification.from_pretrained(
    'DeepPavlov/rubert-base-cased-sentence', 
    num_labels=5,
)

bert_model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [42]:
from transformers import AutoModelForSequenceClassification
auto_model = AutoModelForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased-sentence",
    num_labels=5,
)
auto_model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [43]:
epochs = 2
optimizer = AdamW(bert_model.parameters(), lr=2e-5, eps=1e-8)
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

def train(model, train_dataloader, test_dataloader, epochs, optimizer, scheduler):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_f1 = 0
        for batch in tqdm(train_dataloader):
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, labels = batch
            optimizer.zero_grad()
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs[0]
            train_loss += loss.item()
            loss.backward()
            optimizer.step()
            scheduler.step()
            model.zero_grad()
        train_loss /= len(train_dataloader)
        train_f1 /= len(train_dataloader)
        print(f"Epoch: {epoch+1}")
        print(f"Train Loss: {train_loss}")
        print(f"Train F1: {train_f1}")
        print("Evaluating...")
        model.eval()
        test_loss = 0
        test_f1 = 0
        for batch in tqdm(test_dataloader):
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, labels = batch
            with torch.no_grad():
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )
            loss = outputs[0]
            logits = outputs[1]
            test_loss += loss.item()
            preds = torch.argmax(logits, dim=1).flatten()
            # test_f1 += f1_score(labels, preds, average='macro')
            # use Tensor.CPU() to convert the tensor to a numpy array
            test_f1 += f1_score(labels.cpu(), preds.cpu(), average='macro')
        test_loss /= len(test_dataloader)
        test_f1 /= len(test_dataloader)
        print(f"Test Loss: {test_loss}")
        print(f"Test F1: {test_f1}")


c:\Users\sarta\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [44]:
train(bert_model, train_dataloader, test_dataloader, epochs, optimizer, scheduler)

# save the model
torch.save(bert_model.state_dict(), 'models/bert_model.pt')

  0%|          | 0/335 [00:00<?, ?it/s]

Epoch: 1
Train Loss: 1.0253152303731263
Train F1: 0.0
Evaluating...


  0%|          | 0/84 [00:00<?, ?it/s]

Test Loss: 0.9221071029702822
Test F1: 0.6078717153950917


  0%|          | 0/335 [00:00<?, ?it/s]

Epoch: 2
Train Loss: 0.7521239144588584
Train F1: 0.0
Evaluating...


  0%|          | 0/84 [00:00<?, ?it/s]

Test Loss: 0.9137652023207574
Test F1: 0.6201265198860558


In [45]:
import os

def predict(text) -> str:
    # in theory if you wanna have this function in a separate file
    # you need to inline the functions that are used inside this function
    # and import the BertForSequenceClassification model from the transformers library
    # after this, you can use the func as a standalone
    if clean_text is None:
        raise Exception('clean_text function is not defined go to my github and copy it from there')
    if tokenize is None:
        raise Exception('tokenize function is not defined go to my github and copy it from there')
    text = clean_text(text)
    input_ids, attention_mask = tokenize(text)
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    # load the bert model from the saved state if it's not already loaded
    if not os.path.exists('models/bert_model.pt'):
        raise Exception('the model is not trained yet')
    bert_model.load_state_dict(torch.load('models/bert_model.pt'))
    bert_model.to(device)

    bert_model.eval()
    with torch.no_grad():
        outputs = bert_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
    logits = outputs[0]
    preds = torch.argmax(logits, dim=1).flatten()

    match preds.item():
        case 0:
            return "the sentence is skipped due to low confidence"
        case 1:
            return "positive"
        case 2:
            return "neutral, the text is balanced"
        case 3:
            return "speech, it's a regular text"
        case 4:
            return "negative, ban this user"
        case _:
            return "unknown"

def predict_with_input(text) -> (str, str):
    res = predict(text)
    return f"{text} -> {res}\n"

In [49]:
print(
    predict_with_input("Сегодня был очень хороший день"),
    predict_with_input("Я твой рот в кино водил"),
    predict_with_input("Я не хочу тебя больше видеть"),
    predict_with_input("Собака ты сутулая"),
    predict_with_input("Тварь ты сутулая"),
    predict_with_input("Собака ты сутулая, ты мне нравишься"),
)

Сегодня был очень хороший день -> positive
 Я твой рот в кино водил -> neutral, the text is balanced
 Я не хочу тебя больше видеть -> negative, ban this user
 Собака ты сутулая -> negative, ban this user
 Тварь ты сутулая -> negative, ban this user
 Собака ты сутулая, ты мне нравишься -> positive

